In [ ]:
# average-mean-tabpfn

import os
import pandas as pd

# Path to the folder with CSV files
folder = ""
files = [f for f in os.listdir(folder) if f.endswith('.csv')]

# List to store all DataFrames
dfs = []

# Read and accumulate data
for file in files:
    file_path = os.path.join(folder, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate all files into a single DataFrame
df_total = pd.concat(dfs, ignore_index=True)

# Check if 'threshold' column is present
if 'threshold' not in df_total.columns:
    raise ValueError("The column 'threshold' was not found in the CSV files.")

# Remove non-numeric columns except 'threshold'
numeric_columns = df_total.select_dtypes(include='number').columns.tolist()
if 'threshold' not in numeric_columns:
    numeric_columns.insert(0, 'threshold')

df_total = df_total[numeric_columns]

# Group by 'threshold' and calculate mean and standard deviation
mean_df = df_total.groupby('threshold').mean()
std_df = df_total.groupby('threshold').std()

# Add column indicating the type of statistic
mean_df['fold'] = 'mean'
std_df['fold'] = 'std_dev'

# Merge results vertically
result = pd.concat([mean_df, std_df], axis=0).reset_index()

# Save the result
result.to_csv(os.path.join(folder, 'mlp_oversampling_mean_std.csv'), index=False)


In [ ]:
# average-mean-tabpfn

import os
import pandas as pd

# List of input CSV files
files = [
    "../results_TabPFN_Test_fold0.csv",
    "../results_TabPFN_Test_fold1.csv",
    "../results_TabPFN_Test_fold2.csv",
    "../results_TabPFN_Test_fold3.csv",
    "../results_TabPFN_Test_fold4.csv"
]

# Check if all files exist
for path in files:
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")

dfs = []

# Read each file and add a 'fold' identifier
for i, file_path in enumerate(files):
    df = pd.read_csv(file_path)
    df['fold'] = i
    dfs.append(df)

# Concatenate all files into a single DataFrame
df_total = pd.concat(dfs, ignore_index=True)

# Check if 'Threshold' column is present
if 'Threshold' not in df_total.columns:
    raise ValueError("The column 'Threshold' was not found in the CSV files.")

# Select numeric columns for statistics
numeric_columns = df_total.select_dtypes(include='number').columns.tolist()
columns_for_stats = ['Threshold'] + [col for col in numeric_columns if col != 'fold']

# Calculate mean and standard deviation grouped by 'Threshold'
mean_df = df_total[columns_for_stats].groupby('Threshold').mean().reset_index()
std_df = df_total[columns_for_stats].groupby('Threshold').std().reset_index()

# Add a column indicating the statistic type
mean_df['fold'] = 'mean'
std_df['fold'] = 'std_dev'

# Ensure the same column order
final_columns = df_total.columns.tolist()
mean_df = mean_df[final_columns]
std_df = std_df[final_columns]

# Combine all data (original + mean + std)
final_result = pd.concat([df_total, mean_df, std_df], ignore_index=True)

# Define output folder
output_folder = os.path.dirname(files[0])

# Save final result
final_result.to_csv(os.path.join(output_folder, '../Test_Results_Version2/TabPFN.csv'), index=False)



In [ ]:
# === Statistical Test ===

import pandas as pd
from scipy.stats import friedmanchisquare, f
import scikit_posthocs as sp

# === Config ===
csv_file = "../all_models_statistics_B7.csv"
model_column = "Model"
fold_column = "Fold"
metrics = ["TSS", "HSS", "AUC", "TPR", "TNR", "FPR", "FNR"]
output_csv = "../friedman_iman_nemenyi_holm_results_B7.csv"

# === Read CSV ===
df = pd.read_csv(csv_file, sep=";")

rows = []

for metric in metrics:
    print(f"\n--- Metric: {metric} ---")

    # Reshape into matrix format: each column = model, each row = fold
    table = df.pivot(index=fold_column, columns=model_column, values=metric)

    # Apply Friedman test
    stat, p = friedmanchisquare(*[table[model] for model in table.columns])

    # Parameters
    N = table.shape[0]  # number of folds
    k = table.shape[1]  # number of models

    # Compute Iman–Davenport statistic
    F_F = ((N - 1) * stat) / (N * (k - 1) - stat)
    df1 = k - 1
    df2 = (k - 1) * (N - 1)
    p_davenport = f.sf(F_F, df1, df2)  # p-value

    # Compute Nemenyi post-hoc test
    nemenyi = sp.posthoc_nemenyi_friedman(table.values)
    nemenyi.index = table.columns
    nemenyi.columns = table.columns

    # Compute Dunn-Holm post-hoc test
    holm_results = sp.posthoc_dunn(table.T.values, p_adjust='holm')
    holm_results.index = table.columns
    holm_results.columns = table.columns

    # Generate only unique model comparisons (Model1 < Model2)
    models = table.columns.tolist()
    for i in range(len(models)):
        for j in range(i + 1, len(models)):
            model1 = models[i]
            model2 = models[j]

            p_nemenyi = nemenyi.loc[model1, model2]
            p_holm = holm_results.loc[model1, model2]

            rows.append({
                "Metric": metric,
                "Friedman_Stat": stat,
                "p_value_Friedman": p,
                "ImanDavenport_Stat": F_F,
                "p_value_ImanDavenport": p_davenport,
                "Model1": model1,
                "Model2": model2,
                "p_value_Nemenyi": p_nemenyi,
                "p_value_Holm": p_holm
            })

# Save all results to CSV
pd.DataFrame(rows).to_csv(output_csv, index=False, float_format="%.8f")

print(f"\nFile generated: {output_csv}")

